# WPO 1
[Oefening 3+4](#oef4)
***


In [ ]:
import numpy as np # numerical calculations in python
from scipy.optimize import fmin, fsolve
from scipy.misc import derivative
import matplotlib.pyplot as plt # package for plotting
from matplotlib import rc
rc('text', usetex=True)
from sympy import *

# allow to show plots inline
%matplotlib inline 

<a id="oef4"></a> 
## Oefening 3+4:
Beschouw de transformaties uit vraag 3:

 $
\begin{cases}
     x = R\cos(\theta)\sin(\phi)\\
     y = R\sin(\theta)\sin(\phi)\\
     z = R\cos(\phi)
\end{cases}
$

en

$ \epsilon = \frac{4x^2 + y}{z}$

Gegeven een verzameling van N metingen ($R_{i},\theta_{i},\phi_{i}$) die gaussisch verdeeld zijn rond de theoretische waarden ($R_{t},\theta_{t},\phi_{t}$) zoals hieronder gesimuleerd. Wetende dat de covariantiematrix berekend kan worden door
### $\hat{\tilde{C}}(\hat{\vec{q_{ij}}}) = \frac{1}{N-1} \sum_{k=1}^{N} (q_{ik} - \overline{q}_{i})(q_{jk} - \overline{q}_{j})$
wordt er opnieuw de covariantiematrix $\tilde{C}(x,y,z)$ en de variantie van $\epsilon$ gevraagd.


In [ ]:
pi = 3.141592654
Ris = np.random.normal(30,3,1000);
Tis = np.random.normal(40.*pi/180.,4.*pi/180.,1000);
Pis = np.random.normal(50.*pi/180.,5.*pi/180.,1000);

### Eerst symbolisch

In [ ]:
x,y,z,x2,y2,z2,r,t,p,eps, sigmaR, sigmat, sigmap, sigmax, sigmay, sigmaz = symbols(r'x y z x2 y2 z2 r t p eps \sigma_{R} \sigma_t \sigma_p \sigma_x \sigma_y \sigma_z')


In [ ]:
#definieer de variabeletransformatie
x = #
y = #
z = #
eps = #

In [ ]:
# bepaal de matrices D en Crtp
D = Matrix(###)
Crtp = Matrix(###)

In [ ]:
D

In [ ]:
Crtp

In [ ]:
# bepaal Cxyz
Cxyz = ###

In [ ]:
Cxyz

### Nu numeriek

In [ ]:
#Bepaal een numerieke uitdrukking voor Crtp aan de hand van de formule in de opgave

In [ ]:
# geef ook een numerieke matrix voor D
D_num = ###
N(D_num,3)

In [ ]:
# bepaal uiteindelijk een numerieke uitdrukking voor Cxyz
Cxyz_num = #
N(Cxyz_num,3)